In [1]:
!pip install nltk

In [ ]:
import nltk
nltk.download('all')

In [3]:
!pip install gensim

### 1. 영어 word2vec 만들기

In [6]:
import re
import urllib.request
import zipfile
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize

#### 1) 훈련 데이터 이해하기

In [7]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/09.%20Word%20Embedding/dataset/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x1559bbca0>)


훈련 데이터 파일은 xml 문법으로 작성되어 있어 자연어를 얻기 위해서는 전처리가 필요합니다. 얻고자 하는 실질적 데이터는 영어문장으로만 구성된 내용을 담고 있는 <content>와 </content> 사이의 내용입니다. 전처리 작업을 통해 xml 문법들은 제거하고, 해당 데이터만 가져와야 합니다. 뿐만 아니라, <content>와 </content> 사이의 내용 중에는 (Laughter)나 (Applause)와 같은 배경음을 나타내는 단어도 등장하는데 이 또한 제거해야 합니다.

#### 2) 훈련 데이터 전처리 하기

In [25]:
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.
content_text = re.sub(r'\([^)]*\)', '', parse_text)

# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.
sent_text = sent_tokenize(content_text)

# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.
normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)

# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.
result = [word_tokenize(sentence) for sentence in normalized_text]

In [26]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273380


In [27]:
# 샘플 3개만 출력
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


#### 3) Word2Vec 훈련시키기

In [40]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = Word2Vec(sentences=result, window=5, min_count=5, workers=4, sg=1)   # Skip-gram
model1 = Word2Vec(sentences=result, window=5, min_count=5, workers=4, sg=0)  # CBOW 

- size = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- window = 컨텍스트 윈도우 크기
- min_count = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- workers = 학습을 위한 프로세스 수
- sg = 0은 CBOW, 1은 Skip-gram.

In [48]:
model_result = model.wv.most_similar("man")    # skip-gram
print(model_result)
print('-----------------------------------------------------------------------\n')
model1_result = model1.wv.most_similar("man")  # CBOW
print(model1_result)

[('woman', 0.7396081686019897), ('guy', 0.7281558513641357), ('boy', 0.7062798738479614), ('rabbi', 0.6883109211921692), ('michelangelo', 0.6649431586265564), ('soldier', 0.6605366468429565), ('pianist', 0.6581396460533142), ('psychiatrist', 0.6554989218711853), ('testament', 0.6552859544754028), ('person', 0.6517705321311951)]
-----------------------------------------------------------------------

[('woman', 0.8330977559089661), ('guy', 0.8008432388305664), ('lady', 0.7870305180549622), ('boy', 0.7807929515838623), ('girl', 0.7462363243103027), ('soldier', 0.7294784188270569), ('gentleman', 0.7160805463790894), ('kid', 0.7051184177398682), ('poet', 0.6941379308700562), ('surgeon', 0.680122971534729)]


#### 4) Word2Vec 모델 저장하고 로드하기

In [46]:
model.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

model1.wv.save_word2vec_format('eng_w2v') # 모델 저장
loaded_model1 = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [47]:
# 로드한 모델에 대해서 다시 man과 유사한 단어를 출력
model_result = loaded_model.most_similar("man")
print(model_result)
print('-----------------------------------------------\n')
model1_result = loaded_model1.most_similar("man")
print(model1_result)

[('woman', 0.7396081686019897), ('guy', 0.7281558513641357), ('boy', 0.7062798738479614), ('rabbi', 0.6883109211921692), ('michelangelo', 0.6649431586265564), ('soldier', 0.6605366468429565), ('pianist', 0.6581396460533142), ('psychiatrist', 0.6554989218711853), ('testament', 0.6552859544754028), ('person', 0.6517705321311951)]
-----------------------------------------------

[('woman', 0.8330977559089661), ('guy', 0.8008432388305664), ('lady', 0.7870305180549622), ('boy', 0.7807929515838623), ('girl', 0.7462363243103027), ('soldier', 0.7294784188270569), ('gentleman', 0.7160805463790894), ('kid', 0.7051184177398682), ('poet', 0.6941379308700562), ('surgeon', 0.680122971534729)]


### Use Pre-trained Google Word2vec model

In [4]:
from gensim.models import KeyedVectors
filename = 'GoogleNews-vectors-negative300.bin'
# load the google word2vec model
model = KeyedVectors.load_word2vec_format(filename, binary=True)
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7118192911148071)]


### Doc2vec Train using Gensim

In [18]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from nltk.tokenize import word_tokenize
corpus = [ 'This is the first document.',
'This document is the second document.',
'And this is the third one.',
'Is this the first document?' ]
corpus = [list(word_tokenize(doc)) for doc in corpus]
corpus = [ TaggedDocument(words, ['d{}'.format(idx)]) for idx, words in enumerate(corpus) ]
print("corpus : ",corpus)
model = Doc2Vec(corpus, vector_size=5, min_count=0)
print('----------------------------------------------------------------\n')
print(model.docvecs)
print(model.docvecs[0])
print(model.docvecs[1])
print(model.docvecs[2])
print(model.docvecs[3])


corpus :  [TaggedDocument(words=['This', 'is', 'the', 'first', 'document', '.'], tags=['d0']), TaggedDocument(words=['This', 'document', 'is', 'the', 'second', 'document', '.'], tags=['d1']), TaggedDocument(words=['And', 'this', 'is', 'the', 'third', 'one', '.'], tags=['d2']), TaggedDocument(words=['Is', 'this', 'the', 'first', 'document', '?'], tags=['d3'])]
----------------------------------------------------------------

[-0.10476875 -0.11964148 -0.19767322  0.17072192  0.07122663]
[ 0.00511192 -0.19766597 -0.10348055 -0.1945817   0.04013548]
[ 0.05658225  0.09294482 -0.08611333 -0.06295341 -0.06163875]
[-0.17477198  0.04350599  0.18467224 -0.19060297 -0.06943313]


/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/2580391979.py:12: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs)
/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/2580391979.py:13: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs[0])
/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/2580391979.py:14: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs[1])
/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/2580391979.py:15: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  print(model.docvecs[2])
/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/2580391979.py:16: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `d

In [24]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

cos_list = []
for i in range(4):
  for j in range(4):
    cos_list.append(cos_sim(model.docvecs[i],model.docvecs[j]))
print(cos_list)


[1.0, 0.14078726, -0.29381293, -0.5908979, 0.14078726, 1.0, 0.012296972, 0.0581112, -0.29381293, 0.012296972, 1.0, -0.10139175, -0.5908979, 0.0581112, -0.10139175, 1.0]


/var/folders/rn/7s4510t94j703wxxn5gjx2lc0000gn/T/ipykernel_15614/1854483201.py:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  cos_list.append(cos_sim(model.docvecs[i],model.docvecs[j]))
